In [0]:
from __future__ import print_function

from keras import backend as K

from keras.preprocessing import sequence
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Activation, BatchNormalization
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D

import numpy as np
import pandas as pd
import pylab as plt

In [0]:
# @title Constants

# Embedding
max_features = 20000
maxlen = 100
embedding_size = 128

# Convolution
kernel_size = 5
filters = 64
pool_size = 4

# LSTM
lstm_output_size = 70

# Training
batch_size = 30
epochs = 2

In [0]:
# @title Dataset preparation

x_train = np.zeros((10,100)) # replace with real dataset

# Raw data

# Tokenization

# Lemmatization

# Padding


In [27]:
# @title Base network

txt_in = Input(shape=x_train.shape[1:])
n_layer = Embedding(max_features, embedding_size, input_length=maxlen)(txt_in)
n_layer = Dropout(0.25)(n_layer)
n_layer = Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1)(n_layer)
n_layer = MaxPooling1D(pool_size=pool_size)(n_layer)
n_layer = LSTM(lstm_output_size)(n_layer)

base_model = Model(inputs = [txt_in], outputs = [n_layer], name = 'BaseModelCNN-LSTM')
base_model.summary()

Model: "BaseModelCNN-LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 100, 128)          2560000   
_________________________________________________________________
dropout_4 (Dropout)          (None, 100, 128)          0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 96, 64)            41024     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 24, 64)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 70)                37800     
Total params: 2,638,824
Trainable params: 2,638,824
Non-trainable params: 0
_______________________________________

In [28]:
# @title Siamese network

from keras.layers import concatenate

txt_a_in = Input(shape = x_train.shape[1:], name = 'Text_A_Input')
txt_b_in = Input(shape = x_train.shape[1:], name = 'Text_B_Input')

txt_a_feat = base_model(txt_a_in)
txt_b_feat = base_model(txt_b_in)

combined_features = concatenate([txt_a_feat, txt_b_feat], name = 'merge_features')
combined_features = Dense(16, activation = 'linear')(combined_features)
combined_features = BatchNormalization()(combined_features)
combined_features = Activation('relu')(combined_features)
combined_features = Dense(4, activation = 'linear')(combined_features)
combined_features = BatchNormalization()(combined_features)
combined_features = Activation('relu')(combined_features)
combined_features = Dense(1, activation = 'sigmoid')(combined_features)

hypernimy_model = Model(inputs = [txt_a_in, txt_b_in], outputs = [combined_features], name = 'Hypernimy_Model')
hypernimy_model.summary()

Model: "Hypernimy_Model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Text_A_Input (InputLayer)       (None, 100)          0                                            
__________________________________________________________________________________________________
Text_B_Input (InputLayer)       (None, 100)          0                                            
__________________________________________________________________________________________________
BaseModelCNN-LSTM (Model)       (None, 70)           2638824     Text_A_Input[0][0]               
                                                                 Text_B_Input[0][0]               
__________________________________________________________________________________________________
merge_features (Concatenate)    (None, 140)          0           BaseModelCNN-LSTM[1

In [0]:
# @title Metrics and optimization

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

hypernimy_model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['acc', f1_m, precision_m, recall_m])

In [0]:
# @title Training and validation

print('Train...')
hypernimy_model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))
score, acc = hypernimy_model.evaluate(x_test, y_test, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)